In [ ]:
import gurobipy as gp;
from gurobipy import GRB;

In [ ]:
# Input data
Water=[[100, 100, 150],
       [-75, -75, -50],
       [200, 200, 250],
       [250, 250, 400]];
Prob = [0.4, 0.3, 0.2, 0.1];
w0 = -150;
lb = -250;
floodCost = 20000;
importCost = 10000;
releaseIncome = 6000;
T = 3;
numScens = len(Prob);

meanWater = [0.0]*T; # Careful - this technique doesn't generalize to multidimensional arrays or lists of lists

for t in range(T):
    for k in range(numScens):
        meanWater[t] = meanWater[t]+Water[k][t]*Prob[k];


In [ ]:
m = gp.Model("DAM_MVP");
# Set the sense for the objective: we'd like to maximize
m.modelSense = GRB.MAXIMIZE; # this is to maximize the total profit

In [ ]:
# Amount of water to release each month
x = {};
for i in range(T+1):
    x[i] = m.addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY, obj = releaseIncome);

# water level of each month after flooding, importing and releasing water
wl = {};
for i in range(T+1):
    wl[i] = m.addVar(vtype=GRB.CONTINUOUS, lb = -GRB.INFINITY, ub = 0, obj = 0);
    
# Amount of water to import each month
y = {};
for i in range(T+1):
    y[i] = m.addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY, obj = -importCost);
    
# Amount of water overflow each month
z = {};
for i in range(T+1):
    z[i] = m.addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY, obj = -floodCost);



In [ ]:
# Maintain the water level
for i in range(T+1):
    if i == 0:
        m.addConstr(wl[i]==w0+y[i]-x[i]-z[i]);
        m.addConstr(w0+y[i]-x[i]>=lb);
    else:
        m.addConstr(wl[i]==wl[i-1]+y[i]-x[i]+meanWater[i-1]-z[i]);
        m.addConstr(wl[i-1]+y[i]-x[i]>=lb);
    
# Finish adding all variables and constraints, do an update to make sure things are up to date.
m.update();
m.setParam("OutputFlag", 0); # disable output information unless you want to take a look at the solution process

In [ ]:
m.optimize();
if m.status == GRB.OPTIMAL:
    print('\nOptimal Obj Value: %g' % m.objVal);
    for i in range(T+1):
        print('\nTime[%s]:' % i);
        print('x = %g, y = %g, z = %g, wl = %g' % (m.getAttr('x', x)[i],m.getAttr('x', y)[i],m.getAttr('x', z)[i],m.getAttr('x', wl)[i]));
else:
    print('No solution');

We now solve the multi-stage SP model, create multiple copies of scenario-based variables, one copy per scenario.

In [ ]:
#########################################################
# Now prepare the scenario data
totalScens = pow(numScens,T);
scenProb = [0.0]*totalScens;
scenData = [[0 for j in range(T)] for i in range(totalScens)];

# A lazy way, don't mimic!!
for i1 in range(numScens):
    for i2 in range(numScens):
        for i3 in range(numScens):
            scenProb[i1*pow(numScens,2)+i2*numScens+i3] = Prob[i1]*Prob[i2]*Prob[i3];
            scenData[i1*pow(numScens,2)+i2*numScens+i3][0] = i1;
            scenData[i1*pow(numScens,2)+i2*numScens+i3][1] = i2;
            scenData[i1*pow(numScens,2)+i2*numScens+i3][2] = i3;


In [ ]:
# Now define variables in the Multistage SP model
m_multi= gp.Model("DAM_Multi");
m_multi.modelSense = GRB.MAXIMIZE; # this is to maximize the total profit
# Amount of water to release each month
x_scen = {};
for k in range(totalScens):
    x_scen[k] = {};
    for i in range(T+1):
        x_scen[k][i] = m_multi.addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY, obj=scenProb[k]*releaseIncome);
        
# water level of each month after flooding, importing and releasing water
wl_scen = {};
for k in range(totalScens):
    wl_scen[k] = {};
    for i in range(T+1):
        wl_scen[k][i] = m_multi.addVar(vtype=GRB.CONTINUOUS, lb = -GRB.INFINITY, ub = 0, obj=0);
        
# Amount of water to import each month
y_scen = {};
for k in range(totalScens):
    y_scen[k] = {};
    for i in range(T+1):
        y_scen[k][i] = m_multi.addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY, obj=scenProb[k]*(-importCost));
        
# Amount of water overflow each month
z_scen = {};
for k in range(totalScens):
    z_scen[k] = {};
    for i in range(T+1):
        z_scen[k][i] = m_multi.addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY, obj=scenProb[k]*(-floodCost));

In [ ]:
# Maintain the water level
for k in range(totalScens):
    for i in range(T+1):
        if i == 0:
            m_multi.addConstr(wl_scen[k][i]==w0+y_scen[k][i]-x_scen[k][i]-z_scen[k][i]);
            m_multi.addConstr(w0+y_scen[k][i]-x_scen[k][i]>=lb);
        else:
            m_multi.addConstr(wl_scen[k][i]==wl_scen[k][i-1]+y_scen[k][i]-x_scen[k][i]+Water[scenData[k][i-1]][i-1]-z_scen[k][i]);
            m_multi.addConstr(wl_scen[k][i-1]+y_scen[k][i]-x_scen[k][i]>=lb);

In [ ]:
# Now, define the Nonanticipativity constraints, we define them stage by stage
# WARNING: here I hardcode the problem for T = 3!!! Don't mimic! I am just lazy here
# t = 1

# All scenarios are identical at this point, so the decisions made should also be identical!
for k in range(1,totalScens):
    m_multi.addConstr(x_scen[k][0]==x_scen[0][0]);
    m_multi.addConstr(y_scen[k][0]==y_scen[0][0]);
    m_multi.addConstr(z_scen[k][0]==z_scen[0][0]);
    m_multi.addConstr(wl_scen[k][0]==wl_scen[0][0]);

# t = 2
for tt in range(numScens):
    # There are numScens NAC sets, now construct them one by one
    # Initiate an empty array
    NACindex = [];
    # Now construct the NAC set
    for s in range(totalScens):
        if scenData[s][0] == tt:
            NACindex.append(s);
    s0 = NACindex[0];
    for i in range(1,len(NACindex)):
        s=NACindex[i];
        m_multi.addConstr(x_scen[s][1]==x_scen[s0][1]);
        m_multi.addConstr(y_scen[s][1]==y_scen[s0][1]);
        m_multi.addConstr(wl_scen[s][1]==wl_scen[s0][1]);
        m_multi.addConstr(z_scen[s][1]==z_scen[s0][1]);

# t = 3
for t1 in range(numScens):
    for t2 in range(numScens):
        # There are numScens^2 NAC sets
        NACindex = [];
        for s in range(totalScens):
            if scenData[s][0]==t1 and scenData[s][1]==t2:
                NACindex.append(s);
        s0 = NACindex[0];
        for i in range(1,len(NACindex)):
            s=NACindex[i];
            m_multi.addConstr(x_scen[s][2]==x_scen[s0][2]);
            m_multi.addConstr(y_scen[s][2]==y_scen[s0][2]);
            m_multi.addConstr(wl_scen[s][2]==wl_scen[s0][2]);
            m_multi.addConstr(z_scen[s][2]==z_scen[s0][2]);

# Finish adding all variables and constraints, do an update to make sure things are up to date.
m_multi.update();
m_multi.setParam("OutputFlag", 0); # disable output information unless you want to take a look at the solution process

In [ ]:
m_multi.optimize();
if m_multi.status == GRB.OPTIMAL:
    print('\nOptimal Obj Value: %g' % m_multi.objVal);
else:
    print("Status = ", m_multi.status)
    print('No solution');